In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [2]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [3]:
def ode_model(z, t, beta, gamma):
# no E, no sigma, no incubation
    S, I, R = z
    N = S + I + R
    dSdt = -beta*S*I/N
    dIdt = beta*S*I/N - gamma*I
    dRdt = gamma*I
    return [dSdt, dIdt, dRdt]

In [4]:
def ode_solver(t, initial_conditions, params):
    initI, initR, initN = initial_conditions
    beta, gamma = params
    initS = initN - (initI + initR)
    res = odeint(ode_model, [initS, initI, initR], t, args=(beta, gamma))
    return res

In [5]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
# explain what is R0 and its effect on gamma
initN = 1380000000
initI = 1
initR = 0
gamma = 1/2.9
R0 = 4
beta = R0 * gamma
days = 150

In [6]:
def main(initI, initR, initN, beta, gamma, days):
    initial_conditions = [initI, initR, initN]
    params = [beta, gamma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, I, R = sol[:, 0], sol[:, 1], sol[:, 2]
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))
       
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    # Edit the layout
    fig.update_layout(title='Simulation of SIR Model',xaxis_title='Day', yaxis_title='Counts', title_x=0.5, width=900, height=600)
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    #fig.write_image("seir_simulation.png")
    fig.show()

In [7]:
interact(main, initI=IntSlider(min=0, max=100000, step=10, value=initI, description='initI', style=style, layout=slider_layout),
        initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
        initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
        beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
        gamma=FloatSlider(min=0, max=4, step=0.01, value=gamma, description='Recovery rate', style=style, layout=slider_layout),
        days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout));

interactive(children=(IntSlider(value=1, description='initI', layout=Layout(width='99%'), max=100000, step=10,…